## Run KL analysis for intergenic region, footprinting region, and DHS sites

In [1]:
import sys
import pandas as pd
import torch
import pyro
from tqdm import tqdm


sys.path.insert(0, '/home/djl34/lab_pd/kl/git/KL')

import raklette_updated


/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## load file

In [2]:
KL_data_dir = "/home/djl34/lab_pd/kl/data/"
scratch_dir = "/n/scratch3/users/d/djl34/"

In [3]:
sfs = pd.read_csv(KL_data_dir + "neutral_SFS_5bins.tsv", sep = "\t")

bin_columns = []

for i in range(5):
    bin_columns.append(str(i) + "_bin")

neutral_sfs = torch.tensor(sfs[bin_columns].values)

mu_ref = torch.tensor(sfs["mu"].values)

In [4]:
df = pd.read_csv(scratch_dir + "kl_input/enhancer_module/22.tsv", sep = "\t")

In [5]:
df = df.sort_values("e_module")

In [6]:
df["e_module"] = df["e_module"] + 1

In [7]:
mu_vals = torch.tensor(df["mu_index"].values)
gene_ids = torch.tensor(df["e_module"].values)
gene_ids = gene_ids.type(torch.LongTensor)
sample_sfs = torch.tensor(df["Freq_bin"].values)

In [8]:
mu_vals.shape

torch.Size([77201997])

In [8]:
covariates = torch.tensor([1]*len(mu_vals))
covariates = covariates.unsqueeze(0).transpose(0,1).type(torch.FloatTensor)

## inference portion

In [9]:
#define variables
#define neut_sfs_full, mu_vals, gene_ids, covariates
# n_covs = covariates.shape[-1]          # number of covariates included
n_genes = len(torch.unique(gene_ids)) + 1  # number of genes

n_covs = 0

# n_covs = 1
# n_genes = 3
n_bins = len(neutral_sfs[1]) - 1

In [10]:
#define model and guide
KL = raklette_updated.raklette(neutral_sfs, n_bins, mu_ref, n_covs, n_genes)
model = KL.model
guide = pyro.infer.autoguide.AutoNormal(model)

In [11]:
#run inference
pyro.clear_param_store()
# run SVI
adam = pyro.optim.Adam({"lr":0.005})
elbo = pyro.infer.Trace_ELBO(num_particles=1, vectorize_particles=True)
svi = pyro.infer.SVI(model, guide, adam, elbo)
losses = []

In [ ]:
n_steps = 1000

for step in tqdm(range(n_steps)): # tqdm is just a progress bar thing 
    loss = svi.step(mu_vals, gene_ids, None, sample_sfs)
    print(loss)
    losses.append(loss)

  0%|                                                                                                                                                             | 0/1000 [00:00<?, ?it/s]

In [49]:
# for epoch in range(num_epochs):
#     losses = []

#     # Take a gradient step for each mini-batch in the dataset
#     for x, y in dataloader:
#         if y is not None:
#             y = y.type_as(x)
#         loss = svi.step(x, y)
#         losses.append(loss)

#     # Tell the scheduler we've done one epoch.
#     scheduler.step()

#     print("[Epoch %02d]  Loss: %.5f" % (epoch, np.mean(losses)))

# print("Finished training!")

3647420

In [ ]:
fig, ax = raklette_updated.plot_losses(losses)

## post inference portion (same as raklette_daniel.py)

In [29]:
np.mean(result["KL_fw_post"], axis = 0)

array([7.76460804, 0.37161576, 7.52047177])

In [30]:
np.mean(result["KL_rv_post"], axis = 0)

array([3.98525769, 0.52109279, 3.9743914 ])